## Example showing generalized processing of commuting terms

## Extract Qubit-wise Commuting Groups from a Hamiltonian -  Working Example 


In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from itertools import combinations
import numpy as np

# Observable Helper Functions
from observables import *
    
# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

''' another test
H = SparsePauliOp(['ZZII', 'ZIIZ', 'IZZI', 'IIZZ', 'XIII', 'IXII', 'IIIX', 'IIXI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 2.+0.j, 2.+0.j, 2.+0.j, 2.+0.j])
hamiltonian = [
    ('ZZII', 0.5),
    ('ZIIZ', 0.3),
    ('IZZI', 0.4),
    ('IIZZ', 0.2),
    ('XIII', 0.6),
    ('IXII', 0.1),
    ('IIIX', 0.7),
    ('IIXI', 0.7)
]
'''

groups = group_commuting_terms(hamiltonian)
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    for pauli, coeff in group:
        print(f"  {pauli}: {coeff}")



[SparsePauliOp(['XXII'],
              coeffs=[1.+0.j]), SparsePauliOp(['IYYI'],
              coeffs=[1.+0.j]), SparsePauliOp(['IIZZ'],
              coeffs=[1.+0.j]), SparsePauliOp(['XYII'],
              coeffs=[1.+0.j]), SparsePauliOp(['IIYX'],
              coeffs=[1.+0.j]), SparsePauliOp(['IZXI'],
              coeffs=[1.+0.j]), SparsePauliOp(['XIII'],
              coeffs=[1.+0.j])]
[[False False  True False  True False  True]
 [False False False  True  True False  True]
 [ True False False  True False False  True]
 [False  True  True False  True False  True]
 [ True  True False  True False False  True]
 [False False False False False False  True]
 [ True  True  True  True  True  True False]]
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
Group 1:
  XXII: 0.5
  IIZZ: 0.4
  XIII: 0.7
Group 2:
  IYYI: 0.3
  XYII: 0.

## Compute the energy

In [2]:
from qiskit import QuantumCircuit,transpile, assemble
from qiskit_aer import Aer
import numpy as np

# Initialize the backend and the simulator
backend = Aer.get_backend('qasm_simulator')

'''
groups = [
    [('XXII', 0.5), ('XYII', 0.2), ('XIII', 0.7)],
    [('IYYI', 0.3), ('IIYX', 0.6)],
    [('IIZZ', 0.4), ('IZXI', 0.1)]
]
'''
print("******** processing groups:")
print(groups)

num_qubits = 2

# Function to create circuits for raw Hamiltonian
def create_circuits_ham(ham):
    circuits = []

    for term, coeff in ham:
        print(f"  ... {term}, {coeff}")
        qc = QuantumCircuit(num_qubits)
        for i, pauli in enumerate(term):
            if pauli == 'X':
                qc.h(i)
            elif pauli == 'Y':
                qc.sdg(i)
                qc.h(i)
        qc.measure_all()
        circuits.append((qc, [(term, coeff)]))

    return circuits

# Function to create circuits for each group
def create_circuits(groups):
    circuits = []
    for group in groups:
        qc = QuantumCircuit(num_qubits)

        ## This is NOT correct, it is adding gates for each term; instead it needs to merge the terms and add one for each qubit
        '''
        for term, coeff in group:
            for i, pauli in enumerate(term):
                if pauli == 'X':
                    qc.h(i)
                elif pauli == 'Y':
                    qc.sdg(i)
                    qc.h(i)
         '''
        merged_paulis = ['I'] * num_qubits
        for term, coeff in group:
            for i, pauli in enumerate(term):
                if pauli != "I": merged_paulis[i] = pauli

        print(f"... merged_paulis = {merged_paulis}")

        merged_term = "".join(merged_paulis)
        print(f"... merged_term = {merged_term}")
            
        for i, pauli in enumerate(merged_term):
            if pauli == 'X':
                qc.h(i)
            elif pauli == 'Y':
                qc.sdg(i)
                qc.h(i)
                    
        qc.measure_all()
        circuits.append((qc, group))

    return circuits

# Function to calculate expectation values from results
def calculate_expectation(results, circuits):
    total_energy = 0
    for (qc, group), result in zip(circuits, results.get_counts()):
        #counts = result.get_counts()
        counts = result
        num_shots = sum(counts.values())
        print(f"... got num_shots = {num_shots}: counts = {counts}")
        
        for term, coeff in group:
            print(f"  ... for term: {term}, {coeff}")
            # Calculate the expectation value for each term
            exp_val = 0
            for bitstring, count in counts.items():
                parity = (-1)**sum([int(bitstring[i]) for i, pauli in enumerate(term) if pauli != 'I'])
                exp_val += parity * count
                
            exp_val /= num_shots
            
            total_energy += coeff * exp_val
            print(f"    ... exp_val = {exp_val} {coeff * exp_val}")
            
    return total_energy



# ======================================================

# Define the Hamiltonian terms
H_terms = [
    ('ZI', 0.5),
    ('XX', 0.3),
    ('YY', -0.1),
    #(-0.2, 'ZZ')
]

hamiltonian = H_terms

# optimize by use of commuting groups
use_commuting_groups = False

# Create the circuits

if not use_commuting_groups:
    
    print("\n******** creating circuits from Hamltonian:")
    print(hamiltonian)
    circuits = create_circuits_ham(hamiltonian)
    
else:   
    print("\n******** creating circuits from groups:")
    print(groups)
    circuits = create_circuits(groups)

# print the circuts

for circuit in circuits:
    print(f"\ncircuit = {circuit}")
    for c in circuit:
        #print(f"\n--------\n  c = {c}\n------------\n")
        #print(f"\n  c = ")
        pass
        
#print(circuits)


print("************ compute energy 10 times for these groups:")
#print(circuits)

# Calculate the total energy
for i in range(2):

    print("")
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    for tc in transpiled_circuits:
        print(tc)

    print("")
    
    #qobj = assemble(transpiled_circuits)
    results = backend.run(transpiled_circuits).result()

    total_energy = calculate_expectation(results, circuits)
    print(f"Total Energy: {total_energy}")
    
    print("")


******** processing groups:
[[('XXII', 0.5), ('IIZZ', 0.4), ('XIII', 0.7)], [('IYYI', 0.3), ('XYII', 0.2), ('IIYX', 0.6)], [('IZXI', 0.1)]]

******** creating circuits from Hamltonian:
[('ZI', 0.5), ('XX', 0.3), ('YY', -0.1)]
  ... ZI, 0.5
  ... XX, 0.3
  ... YY, -0.1

circuit = (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000002500357D250>, [('ZI', 0.5)])

circuit = (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000025034DB10D0>, [('XX', 0.3)])

circuit = (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000002507CF2CC90>, [('YY', -0.1)])
************ compute energy 10 times for these groups:

         ░ ┌─┐   
   q_0: ─░─┤M├───
         ░ └╥┘┌─┐
   q_1: ─░──╫─┤M├
         ░  ║ └╥┘
meas: 2/════╩══╩═
            0  1 
        ┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas: 2/═════════╩══╩═
                 0  1 
        ┌───────────┐ ░ ┌─┐   
   q_0: ┤ U2(0,π/2) ├─░─┤M├───
        ├────

# Other Examples that do not work

In [3]:



from qiskit.quantum_info import Pauli
import itertools

# Define the Hamiltonian terms
terms = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

# Create Pauli objects
pauli_terms = [Pauli(term[0]) for term in terms]

def commute(pauli_a, pauli_b):
    """Check if two Pauli operators commute."""
    return pauli_a.commutes(pauli_b)  # Corrected method call

def find_commuting_sets(pauli_terms):
    """Find all maximal commuting sets of Pauli operators."""
    commuting_groups = []
    n = len(pauli_terms)

    # Check all combinations of Pauli terms
    for i in range(1, n + 1):
        for combination in itertools.combinations(pauli_terms, i):
            # Check if all terms in the combination commute
            if all(commute(combination[j], combination[k]) for j in range(len(combination)) for k in range(j + 1, len(combination))):
                commuting_groups.append(combination)

    # Filter for maximal sets
    maximal_commuting_sets = []
    for group in commuting_groups:
        is_maximal = True
        for other_group in commuting_groups:
            if group != other_group and set(group).issubset(set(other_group)):
                is_maximal = False
                break
        if is_maximal:
            maximal_commuting_sets.append(group)

    return maximal_commuting_sets

# Find the maximal commuting sets
maximal_commuting_sets = find_commuting_sets(pauli_terms)

# Print the results
for idx, group in enumerate(maximal_commuting_sets):
    print(f"Maximal Commuting Set {idx + 1}: {[str(p) for p in group]}")
    

Maximal Commuting Set 1: ['IYYI', 'IZXI', 'XIII']
Maximal Commuting Set 2: ['XXII', 'IIZZ', 'IIYX', 'XIII']
Maximal Commuting Set 3: ['IYYI', 'XYII', 'IIYX', 'XIII']
Maximal Commuting Set 4: ['IIZZ', 'XYII', 'IIYX', 'XIII']


In [4]:
from qiskit.quantum_info import Pauli
import itertools

# Define the Hamiltonian terms
terms = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

# Create Pauli objects
pauli_terms = [Pauli(term[0]) for term in terms]

def commute(pauli_a, pauli_b):
    """Check if two Pauli operators commute."""
    return pauli_a.commutes(pauli_b)

def find_maximal_commuting_sets(pauli_terms):
    """Find all maximal commuting sets of Pauli operators."""
    n = len(pauli_terms)
    maximal_sets = []

    # Iterate through all combinations of Pauli terms
    for i in range(1, n + 1):
        for combination in itertools.combinations(pauli_terms, i):
            # Check if all terms in the combination commute
            if all(commute(combination[j], combination[k]) for j in range(len(combination)) for k in range(j + 1, len(combination))):
                # Check if the combination is maximal
                is_maximal = True
                for term in pauli_terms:
                    if term not in combination:
                        # Check if the term commutes with all in the current combination
                        if all(commute(term, c) for c in combination):
                            is_maximal = False
                            break
                if is_maximal:
                    maximal_sets.append(set(combination))

    return maximal_sets

# Find the maximal commuting sets
maximal_commuting_sets = find_maximal_commuting_sets(pauli_terms)

# Print the results
for idx, group in enumerate(maximal_commuting_sets):
    print(f"Maximal Commuting Set {idx + 1}: {[str(p) for p in group]}")


Maximal Commuting Set 1: ['IYYI', 'IZXI', 'XIII']
Maximal Commuting Set 2: ['IIYX', 'XXII', 'IIZZ', 'XIII']
Maximal Commuting Set 3: ['IYYI', 'IIYX', 'XIII', 'XYII']
Maximal Commuting Set 4: ['IIYX', 'IIZZ', 'XIII', 'XYII']


In [5]:

from qiskit.quantum_info import Pauli
import itertools

# Define the Hamiltonian terms
terms = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

# Create Pauli objects
pauli_terms = [Pauli(term[0]) for term in terms]

def commute(pauli_a, pauli_b):
    """Check if two Pauli operators commute."""
    return pauli_a.commutes(pauli_b)

def find_maximal_commuting_sets(pauli_terms):
    """Find all maximal commuting sets of Pauli operators."""
    maximal_sets = []
    n = len(pauli_terms)

    # Iterate through all combinations of Pauli terms
    for i in range(1, n + 1):
        for combination in itertools.combinations(pauli_terms, i):
            # Check if all terms in the combination commute
            if all(commute(combination[j], combination[k]) for j in range(len(combination)) for k in range(j + 1, len(combination))):
                # Check if this set can be expanded (maximality check)
                is_maximal = True
                for term in pauli_terms:
                    if term not in combination:
                        # Check if the term commutes with all terms in the current combination
                        if all(commute(term, c) for c in combination):
                            is_maximal = False
                            break
                
                # If it is maximal, add to the list
                if is_maximal:
                    maximal_sets.append(set(combination))

    # Remove duplicates by converting to a set of frozensets (to make it hashable)
    unique_maximal_sets = []
    seen = set()
    for group in maximal_sets:
        frozenset_group = frozenset(group)
        if frozenset_group not in seen:
            unique_maximal_sets.append(group)
            seen.add(frozenset_group)

    return unique_maximal_sets

# Find the maximal commuting sets
maximal_commuting_sets = find_maximal_commuting_sets(pauli_terms)

# Print the results
for idx, group in enumerate(maximal_commuting_sets):
    print(f"Maximal Commuting Set {idx + 1}: {[str(p) for p in group]}")



Maximal Commuting Set 1: ['IYYI', 'IZXI', 'XIII']
Maximal Commuting Set 2: ['IIYX', 'XXII', 'IIZZ', 'XIII']
Maximal Commuting Set 3: ['IYYI', 'IIYX', 'XIII', 'XYII']
Maximal Commuting Set 4: ['IIYX', 'IIZZ', 'XIII', 'XYII']


In [6]:

from qiskit.quantum_info import Pauli
import itertools

# Define the Hamiltonian terms
terms = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

# Create Pauli objects
pauli_terms = [Pauli(term[0]) for term in terms]

def commute(pauli_a, pauli_b):
    """Check if two Pauli operators commute."""
    return pauli_a.commutes(pauli_b)

def find_maximal_commuting_sets(pauli_terms):
    """Find all maximal commuting sets of Pauli operators."""
    n = len(pauli_terms)
    maximal_sets = []

    # Iterate through all combinations of Pauli terms
    for i in range(1, n + 1):
        for combination in itertools.combinations(pauli_terms, i):
            # Check if all terms in the combination commute
            if all(commute(combination[j], combination[k]) for j in range(len(combination)) for k in range(j + 1, len(combination))):
                current_set = set(combination)
                
                # Check for maximality
                is_maximal = True
                for term in pauli_terms:
                    if term not in current_set:
                        # Check if the term commutes with all in the current combination
                        if all(commute(term, c) for c in current_set):
                            is_maximal = False
                            break
                
                if is_maximal:
                    maximal_sets.append(current_set)

    # Remove duplicates
    unique_maximal_sets = []
    seen = set()
    for group in maximal_sets:
        frozenset_group = frozenset(group)
        if frozenset_group not in seen:
            unique_maximal_sets.append(group)
            seen.add(frozenset_group)

    return unique_maximal_sets

# Find the maximal commuting sets
maximal_commuting_sets = find_maximal_commuting_sets(pauli_terms)

# Print the results
for idx, group in enumerate(maximal_commuting_sets):
    print(f"Maximal Commuting Set {idx + 1}: {[str(p) for p in group]}")


Maximal Commuting Set 1: ['IYYI', 'IZXI', 'XIII']
Maximal Commuting Set 2: ['IIYX', 'XXII', 'IIZZ', 'XIII']
Maximal Commuting Set 3: ['IYYI', 'IIYX', 'XIII', 'XYII']
Maximal Commuting Set 4: ['IIYX', 'IIZZ', 'XIII', 'XYII']


In [7]:
from qiskit.quantum_info import Pauli
import itertools

# Define the Hamiltonian terms
terms = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

# Create Pauli objects
pauli_terms = [Pauli(term[0]) for term in terms]

def commute(pauli_a, pauli_b):
    """Check if two Pauli operators commute."""
    return pauli_a.commutes(pauli_b)

def find_maximal_commuting_sets(pauli_terms):
    """Find all maximal commuting sets of Pauli operators."""
    maximal_sets = []
    n = len(pauli_terms)

    # Iterate through all combinations of Pauli terms
    for i in range(1, n + 1):
        for combination in itertools.combinations(pauli_terms, i):
            # Check if all terms in the combination commute
            if all(commute(combination[j], combination[k]) for j in range(len(combination)) for k in range(j + 1, len(combination))):
                current_set = set(combination)
                
                # Check for maximality
                is_maximal = True
                for term in pauli_terms:
                    if term not in current_set:
                        # Check if the term commutes with all terms in the current combination
                        if all(commute(term, c) for c in current_set):
                            is_maximal = False
                            break
                
                # If it is maximal, add to the list
                if is_maximal:
                    maximal_sets.append(current_set)

    # Remove duplicates
    unique_maximal_sets = []
    seen = set()
    for group in maximal_sets:
        frozenset_group = frozenset(group)
        if frozenset_group not in seen:
            unique_maximal_sets.append(group)
            seen.add(frozenset_group)

    return unique_maximal_sets

# Find the maximal commuting sets
maximal_commuting_sets = find_maximal_commuting_sets(pauli_terms)

# Print the results
for idx, group in enumerate(maximal_commuting_sets):
    print(f"Maximal Commuting Set {idx + 1}: {[str(p) for p in group]}")


Maximal Commuting Set 1: ['IYYI', 'IZXI', 'XIII']
Maximal Commuting Set 2: ['IIYX', 'XXII', 'IIZZ', 'XIII']
Maximal Commuting Set 3: ['IYYI', 'IIYX', 'XIII', 'XYII']
Maximal Commuting Set 4: ['IIYX', 'IIZZ', 'XIII', 'XYII']
